- 1. hvor lange tid tage det og ankomme til destination


- 2. Hvor lange tid tage det i genmesnigt for en fire case


- 3. Zipcode Battalion station area


- 4. Suppression Units and Personnel


- 5. Primary Situation


- 6. Action Taken Primary


- 7. Action Taken Primary


- 8. neighborhood_district


- 9. Point for maps


In [88]:
import pandas as pd
df = pd.read_csv("../Fire_Incidents.csv")
df['Incident Date'] = pd.to_datetime(df['Incident Date'])

# Get the data from date 2010-04-01 to 2023-04-01
df = df[(df['Incident Date'] >= '2010-04-01') &
        (df['Incident Date'] <= '2023-04-01')]
df['Incident Time'] = df['Incident Date'].dt.time

# print the number of rows
num_rows = df.shape[0]
print("The number of rows is:", num_rows)
# print(df.head())


C:\Users\salim\AppData\Local\Temp\ipykernel_32684\3851108806.py:2: DtypeWarning: Columns (10,12,13,20,38,45,46,55,59) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../Fire_Incidents.csv")


The number of rows is: 409936


In [89]:
# get just the data from Sf city
df = df[(df['City'] == 'SF') | (df['City'] == 'San Francisco')
        | (df['City'] == 'SAN FRANCISCO')]

# Deleting all unnaseccary columns
df.drop(columns=['Automatic Extinguishing System Present', 'Automatic Extinguishing Sytem Type', 'Automatic Extinguishing Sytem Perfomance',
        'Automatic Extinguishing Sytem Failure Reason', 'Number of Sprinkler Heads Operating',
        'Detectors Present','Detector Type','Detector Operation','Detector Effectiveness','Detector Failure Reason'], inplace=True)


In [90]:
# add Situation_by_code
df['focuse_Situation_by_number'] = df['Primary Situation'].str[:3]
#df.head()

In [93]:
call_Situation = df['Primary Situation']
print(call_Situation)
len(call_Situation)
ListOfSituation = call_Situation.unique()
print(ListOfSituation)
# by using unique() fun we can se the diffrenet type of data



213988               700 - False alarm or false call, other
213989               700 - False alarm or false call, other
213990       324 - Motor vehicle accident with no injuries.
213991               118 - Trash or rubbish fire, contained
213992                 322 - Vehicle accident with injuries
                                ...                        
624510                              522 Water or steam leak
624515    743 Smoke detector activation, no fire - unint...
624518                 700 False alarm or false call, other
624520     733 Smoke detector activation due to malfunction
624524    745 Alarm system activation, no fire - uninten...
Name: Primary Situation, Length: 399396, dtype: object
['700 - False alarm or false call, other'
 '324 - Motor vehicle accident with no injuries.'
 '118 - Trash or rubbish fire, contained'
 '322 - Vehicle accident with injuries'
 '711 - Municipal alarm system, Street Box False'
 '113 - Cooking fire, confined to container' '511 - Lock-out'

In [92]:
Situation_count = call_Situation.value_counts()
# Situation_count.to_csv('Situation_count.csv')
top20 = Situation_count.nlargest(20)  # Get the top 20 most frequent situations
print(top20)

call_Situation_by_number = df['focuse_Situation_by_number']
Situation_count_by_number =call_Situation_by_number.value_counts()
top20_by_number = Situation_count_by_number.nlargest(20)  # Get the top 20 most frequent situations
print(top20_by_number)

700 False alarm or false call, other                      33245
711 - Municipal alarm system, Street Box False            20731
745 Alarm system activation, no fire - unintentional      15983
711 Municipal alarm system, malicious false alarm         14289
500 Service Call, other                                   13672
735 Alarm system sounded due to malfunction               13596
743 Smoke detector activation, no fire - unintentional    12668
322 Motor vehicle accident with injuries                  11752
151 Outside rubbish, trash or waste fire                   9307
554 Assist invalid                                         8792
700 - False alarm or false call, other                     7944
113 Cooking fire, confined to container                    7856
733 Smoke detector activation due to malfunction           7749
745 - Alarm system sounded/no fire-accidental              6981
311 Medical assist, assist EMS crew                        6337
735 - Alarm system sounded due to malfun